In [6]:
# ==========================================
# Notebook: find_best_params.ipynb
# Purpose: Manual Grid Search for ARIMA, SARIMA, SARIMAX
# Stocks: ["AEP", "DUK", "SO", "ED", "EXC"]
# Exog: {"oil": "CL=F", "gas": "NG=F", "xlu": "XLU"}
# ==========================================

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import yfinance as yf
import itertools
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error

# -----------------------------
# Config
# -----------------------------
TICKERS = ["AEP", "DUK", "SO", "ED", "EXC"]
EXOG_TICKERS = {"oil": "CL=F", "gas": "NG=F", "xlu": "XLU"}
PERIOD = "5y"

# -----------------------------
# Helpers
# -----------------------------
def fetch_series(symbol, period="5y"):
    s = yf.download(symbol, period=period, progress=False, auto_adjust=True)["Close"].dropna()
    return s

def fetch_exog(period="5y"):
    exog = pd.DataFrame()
    for name, tkr in EXOG_TICKERS.items():
        try:
            s = yf.download(tkr, period=period, progress=False, auto_adjust=True)["Close"].dropna()
            exog[name] = s
        except:
            pass
    return exog

def evaluate_forecast(true, pred):
    return mean_absolute_error(true, pred)

# -----------------------------
# Grid Search
# -----------------------------
def grid_search_arima(series, p_values, d_values, q_values):
    best_score, best_cfg = float("inf"), None
    for order in itertools.product(p_values, d_values, q_values):
        try:
            model = ARIMA(series, order=order).fit()
            fc = model.forecast(steps=5)
            mae = evaluate_forecast(series[-5:], fc)
            if mae < best_score:
                best_score, best_cfg = mae, order
        except:
            continue
    return best_cfg, best_score

def grid_search_sarima(series, p_values, d_values, q_values, P_values, D_values, Q_values, m):
    best_score, best_cfg = float("inf"), None
    for order in itertools.product(p_values, d_values, q_values):
        for seas in itertools.product(P_values, D_values, Q_values, [m]):
            try:
                model = SARIMAX(series, order=order, seasonal_order=seas,
                                enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
                fc = model.forecast(steps=5)
                mae = evaluate_forecast(series[-5:], fc)
                if mae < best_score:
                    best_score, best_cfg = mae, (order, seas)
            except:
                continue
    return best_cfg, best_score

def grid_search_sarimax(series, exog, p_values, d_values, q_values, P_values, D_values, Q_values, m):
    best_score, best_cfg = float("inf"), None
    for order in itertools.product(p_values, d_values, q_values):
        for seas in itertools.product(P_values, D_values, Q_values, [m]):
            try:
                model = SARIMAX(series, order=order, seasonal_order=seas,
                                exog=exog, enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
                fc = model.forecast(steps=5, exog=exog.iloc[-5:])
                mae = evaluate_forecast(series[-5:], fc)
                if mae < best_score:
                    best_score, best_cfg = mae, (order, seas)
            except:
                continue
    return best_cfg, best_score

# -----------------------------
# Run for each ticker
# -----------------------------
p = range(0, 4)   # 0–3
d = range(0, 3)   # 0–2
q = range(0, 4)   # 0–3
P = range(0, 3)   # 0–2
D = range(0, 2)   # 0–1
Q = range(0, 3)   # 0–2
m = 20            # seasonality (≈ 1 เดือน trading days)

results = {}

for t in TICKERS:
    print("="*50)
    print(f"🔎 {t}")
    s = fetch_series(t, PERIOD)

    # ARIMA
    best_arima, score_a = grid_search_arima(s, p, d, q)
    print(f"ARIMA best: order={best_arima}, MAE={score_a:.4f}")

    # SARIMA
    best_sarima, score_sa = grid_search_sarima(s, p, d, q, P, D, Q, m)
    print(f"SARIMA best: order={best_sarima[0]}, seasonal_order={best_sarima[1]}, MAE={score_sa:.4f}")

    # SARIMAX
    exog = fetch_exog(PERIOD).reindex(s.index).ffill()
    best_sarimax, score_sx = grid_search_sarimax(s, exog, p, d, q, P, D, Q, m)
    print(f"SARIMAX best: order={best_sarimax[0]}, seasonal_order={best_sarimax[1]}, MAE={score_sx:.4f}")

    results[t] = {
        "arima": {"order": best_arima},
        "sarima": {"order": best_sarima[0], "seasonal_order": best_sarima[1]},
        "sarimax": {"order": best_sarimax[0], "seasonal_order": best_sarimax[1]}
    }

print("\n\n✅ Copy ไปใส่ใน MODEL_PARAMS ของ forecast.py")
print(results)


🔎 AEP


c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Pred

ARIMA best: order=(3, 0, 0), MAE=1.2816


c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results w

SARIMA best: order=(2, 0, 1), seasonal_order=(1, 0, 2, 20), MAE=1.2392


TypeError: 'NoneType' object is not subscriptable